In [1]:
from preproc_tabular import get_tabular
from dataset_sales import getDataset
import sales_prediction as sp
from torch.optim.adamw import AdamW
from torch import nn
import torch
import os

img_path = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\ResizedImages\\'
tab_path = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\sales_anagrafica_final.csv'
aggregated_path = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\aggregated_sales_season.csv'
aggregated_month_path = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\aggregated_sales_month.csv'

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Sto usando {device}")
modello = sp.create_model(0).to(device)

batch_size = 128
epochs = 5
criterion = nn.MSELoss()
optimizer = AdamW(modello.parameters(), lr=1e-2) 
early_stop = sp.EarlyStopping(1, 0.001)

data, references, descriptions, target = get_tabular(tab_path)
train, val = getDataset(references, data, descriptions, target, img_path, batch_size, 0.08)

mse_final, mae_final, r2_final = 0, 0, 0
modello_final = 0
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    sp.train_loop(train, modello, criterion, optimizer, batch_size, device)
    mse, mae, r2 = sp.validation_loop(val, modello, criterion, device)
    early_stp = early_stop(mse)
    print(early_stp)
    if early_stp:
        print('Early Stop attivato.')
        break
    mse_final, mae_final, r2_final = mse, mae, r2
    modello_final = modello
    
print("Done!")

os.makedirs('C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\results\\no_agg')
with open('C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\results\\no_agg\\descrizione.txt', 'w') as file:
    file.write("Addestramento con dati non aggregati adamw, batch 128, 5 epoche, lr 1e-2, early stopping con 1 epoca e delta 0.001, dropout 0.5." + '\n\n')
    file.write(f"Metriche alla fine:\nMSE: {mse_final}\nMAE: {mae_final}\nR2: {r2_final}")
model_sc = torch.jit.script(modello_final)
model_sc.save('C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\results\\no_agg\\weights.pt')

c:\Users\GRVRLD00P\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Sto usando cuda


c:\Users\GRVRLD00P\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(


   CodiceColore  PianoTaglia  AreaCode  CategoryCode  SectorCode  \
0     -0.882508    -2.727329 -2.346624     -2.293852   -2.275914   
1     -0.882508    -2.727329 -2.346624     -2.293852   -2.275914   
2     -0.882508    -2.727329 -2.346624     -2.293852   -2.275914   
3     -0.882508    -2.727329 -2.346624     -2.293852   -2.275914   
4     -0.882508    -2.727329 -2.346624     -2.293852   -2.275914   

   DepartmentCode  WaveCode  AstronomicalSeasonExternalID  \
0       -2.275319 -1.461577                     -1.020106   
1       -2.275319 -1.461577                     -1.020106   
2       -2.275319 -1.461577                     -1.020106   
3       -2.275319 -1.461577                     -1.020106   
4       -2.275319 -1.461577                     -1.020106   

   SalesSeasonDescription  LocationId      Week  
0               -0.787985   -0.418528 -1.446871  
1               -0.787985    0.334745 -0.627256  
2               -0.787985   -0.257112 -0.490653  
3               -0.78798

c:\Users\GRVRLD00P\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\functional.py:5560: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)


MSE: 3.115098  [  128/111298]


In [7]:
import pred_tabular as pt
import sales_prediction as sp
from preproc_tabular import get_tabular
from torch.optim.adamw import AdamW
from torch import nn
import torch
import os

img_path = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\ResizedImages\\'
tab_path = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\sales_anagrafica_final.csv'
aggregated_path = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\aggregated_sales_season.csv'
aggregated_month_path = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\aggregated_sales_month.csv'

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Sto usando {device}")

modello = pt.create_model(1).to(device)

batch_size = 128
epochs = 10
criterion = nn.MSELoss()
optimizer = AdamW(modello.parameters(), lr=1e-2) 
early_stop = sp.EarlyStopping(1, 0.001) 

data, _ , _, target = get_tabular(aggregated_path)
train, val = pt.getDataset(data, target, batch_size, 0.3)
print(data.head())

mse_final, mae_final, r2_final = 0, 0, 0
modello_final = 0
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    pt.train_loop(train, modello, criterion, optimizer, batch_size, device)
    mse, mae, r2 = pt.validation_loop(val, modello, criterion, device)
    early_stp = early_stop(mse)
    print(early_stp)
    if early_stp:
        print('Early Stop attivato.')
        break
    mse_final, mae_final, r2_final = mse, mae, r2
    modello_final = modello
    
print("Done!")

#os.makedirs('C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\results\\Tabular_Agg')
with open('C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\results\\Tab_Agg\\descrizione.txt', 'w') as file:
    file.write("Addestramento con dati aggregati 0.3 utilizzando solo i tabular, adamw, batch 128, 10 epoche, lr 1e-2, early stopping con 1 epoca e delta 0.001." + '\n\n')
    file.write(f"Metriche alla fine:\nMSE: {mse_final}\nMAE: {mae_final}\nR2: {r2_final}")

Sto usando cuda
   CodiceColore  PianoTaglia  AreaCode  CategoryCode  SectorCode  \
0      -0.86386    -2.196063 -1.057982     -1.002886   -0.978144   
1      -0.86386    -2.196063 -1.057982     -1.002886   -0.978144   
2      -0.86386    -2.196063 -1.057982     -1.002886   -0.978144   
3      -0.86386    -2.196063 -1.057982     -1.002886   -0.978144   
4      -0.86386    -2.196063 -1.057982     -1.002886   -0.978144   

   DepartmentCode  WaveCode  AstronomicalSeasonExternalID  \
0       -1.003347 -1.297348                     -1.424692   
1       -1.003347 -1.297348                     -1.424692   
2       -1.003347 -1.297348                     -1.424692   
3       -1.003347 -1.297348                     -1.424692   
4       -1.003347 -1.297348                     -1.424692   

   SalesSeasonDescription  LocationId  
0               -1.354523   -1.742905  
1               -1.354523   -1.742905  
2               -1.354523   -1.742905  
3               -1.354523   -1.729290  
4       

RuntimeError: mat1 and mat2 shapes cannot be multiplied (128x10 and 11x11)